In [1]:
import tensorflow as tf
import numpy as np

D:\Anaconda\Initial\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [3]:
#tf.matmul即矩阵相乘，满足前面的列数等于后面的行数即可
a = tf.constant([1, 2, 3, 4, 5, 6], shape=[2, 3]) # [[1. 2. 3.],[4. 5. 6.]]
b = tf.constant([7, 8, 9, 10, 11, 12], shape=[3, 2]) # [[7. 8.],[9. 10.],[11. 12.]]
c = tf.matmul(a, b) # [[58 64],[139 154]]
print(sess.run(c))

[[ 58  64]
 [139 154]]


In [4]:
#tf.multiply即矩阵对应元素相乘，要求两个矩阵必须同等大小
x = tf.constant([[1.0,2.0,3.0],[1.0,2.0,3.0],[1.0,2.0,3.0]])  
y = tf.constant([[1.,0,0],[0,1,0],[0,0,1]])
z = tf.multiply(x,y) 
print(sess.run(z))

[[1. 0. 0.]
 [0. 2. 0.]
 [0. 0. 3.]]


In [5]:
#特殊的，应用于词向量的余弦相似度上
#word2vec中求若干个词向量与总的词向量中相关的
data = np.random.randint(0,10,(1000,120)) #这里表示共1000个词语或字，每个词语或字用120维的一维特征向量表示
data = data.astype(np.float32)
norm = tf.sqrt(tf.reduce_sum(tf.square(data), 1, keep_dims=True)) # 对每行元素进行平方和后开方，主要为了单位化每一行向量
normalized_embeddings = data / norm #相当于对1000个向量，每个向量都做了单位化，主要是为了后面求余弦时，可以直接等于向量点积
print(sess.run(normalized_embeddings))
valid_examples = np.random.choice(100, 10, replace=False) #即从[0,100)里随机不重复挑选10个数
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset) #从总表中挑选出对应数字的十行
#这里表示我随机从总表中随机挑选出10个词语

Instructions for updating:
keep_dims is deprecated, use keepdims instead
[[0.11377978 0.09752552 0.14628828 ... 0.08127127 0.14628828 0.01625425]
 [0.09573385 0.         0.09573385 ... 0.06382257 0.12764513 0.11168949]
 [0.03243575 0.11352513 0.11352513 ... 0.14596088 0.04865362 0.08108938]
 ...
 [0.11760139 0.10080119 0.10080119 ... 0.08400099 0.15120178 0.        ]
 [0.03440105 0.03440105 0.0688021  ... 0.03440105 0.0688021  0.01720052]
 [0.         0.         0.06568862 ... 0.04926647 0.09853294 0.13137725]]


In [6]:
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
#首先这是求的十个字与总表中所有字进行余弦相似度计算
#这里valid_embeddings是从总表中随机挑取的十个字对应的字或者词向量，shape= 10 * 120
#这里normalized_embeddings即总表，代表所有词或字的向量表，shape=1000 * 120

#tf.matmul，我们知道是矩阵相乘，transpose_a和transpose_b默认都为False，第一个代表a是否要转置，第二个代表b是否要转置
#因为是矩阵相乘，故而这里b需要转置，从而使得前一个矩阵列数等于后一个矩阵行数

#两个字或者两个词是否相似，我们用这两个字或词的向量进行求余弦可得，即余弦=(向量a与向量b的点积)/(向量a和向量b的模长之积)
#两个向量点积即对应元素相乘相加，若是单位向量的话，则分母为1，从而余弦=两个向量对应位置元素相乘相加

#这里我们有十个字/词的向量，还有整个字典的字/词向量,而十个字/词是从整个字典中随机取来的，整个词典中每个字/词都是单位向量
#valid_embeddings 与 normalized_embeddings的转置 两矩阵相乘，即 10*120 与 120*1000 相乘，得到10*1000的矩阵
#理解一下，similarity矩阵中每一个元素即是valid_embeddings矩阵中的一个字与normalized_embeddings矩阵中的一个字做的点积
#故而similarity矩阵中每一个元素都相当于是两个字/词的点积结果，即用10个字/词与1000个字/词进行求余弦相似度了。
#值越大，也就是越接近于1的，代表这两个矩阵中有相似的词。
#比如similarity矩阵中第i行第j列元素值为1，其结果是valid_embeddings中第i行与normalized_embeddings第j列相乘的结果
#即代表valid_embeddings的第i个字/词与normalized_embeddings的第j个字/词相似。

#故而要想知道在总表中，与valid_embeddings中第i个字相似的，则去similarity矩阵中的第i行寻找。
#值越大的，代表越相似，并且，该词处于similarity中第j列，则代表是总表normalized_embeddings中第j个字与其相似的

In [7]:
#若只是两个向量求余弦的话
import tensorflow as tf
def cosine_1(a,b):
    length_a = tf.sqrt(tf.reduce_sum(tf.square(a), 1, keepdims=True))
    length_b = tf.sqrt(tf.reduce_sum(tf.square(b), 1, keepdims=True))
    norm_a = a /(length_a+1e-6)
    norm_b = b /(length_b+1e-6)
    return tf.multiply(norm_a,norm_b)

def cosine_2(a,b):
    length_a = tf.sqrt(tf.reduce_sum(tf.square(a), 1, keepdims=True))
    length_b = tf.sqrt(tf.reduce_sum(tf.square(b), 1, keepdims=True))
    return tf.multiply(a,b)/(length_a*length_b+1e-6)

In [8]:
import numpy as np
a = np.array([[1,2],[3,4]],dtype=np.float32)
b = np.array([[5,6],[7,8]],dtype=np.float32)
res_1 = cosine_1(a,b)
res_2 = cosine_2(a,b)
sess = tf.Session()
print(sess.run(res_1),"\n","="*20)
print(sess.run(res_2))

[[0.286299   0.68711764]
 [0.39510265 0.60206115]] 
[[0.2862991  0.6871179 ]
 [0.39510277 0.60206133]]
